In [ ]:
!pip install tensorflow

In [1]:
import tokenization
import tensorflow as tf
import random

from create_pretraining_data import create_training_instances
from types import SimpleNamespace

FLAGS = SimpleNamespace(
    input_file="sample_text.txt",
    vocab_file="uncased_L-2_H-128_A-2/vocab.txt",
    do_lower_case=True,
    random_seed=12345,
    max_seq_length=128,
    max_predictions_per_seq=20,
    dupe_factor=10,
    masked_lm_prob=0.15,
    do_whole_word_mask=False,
    short_seq_prob=0.1
)

print(FLAGS.vocab_file)

uncased_L-2_H-128_A-2/vocab.txt


In [14]:
tokenizer = tokenization.FullTokenizer(
    vocab_file=FLAGS.vocab_file, do_lower_case=FLAGS.do_lower_case)

input_files = []
for input_pattern in FLAGS.input_file.split(","):
    input_files.extend(tf.io.gfile.glob(input_pattern))

print("*** Reading from input files ***")
for input_file in input_files:
    print("  %s", input_file)
    
rng = random.Random(FLAGS.random_seed)

instances = create_training_instances(
    input_files, tokenizer, FLAGS.max_seq_length, FLAGS.dupe_factor,
    FLAGS.short_seq_prob, FLAGS.masked_lm_prob, FLAGS.max_predictions_per_seq, rng)

for i in instances[4].tokens:
    print(i, end=' ')
    if i == '.':
        print()

# wood pile refused to kindle a fire to dry his bed-clothes, and he had recourse to a more provident neighbor's to supply the deficiency.
# This was nearly opposite.
# Mr. Cassius crossed the highway, and stopped suddenly.
# Something glittered in the nearest red pool before him.
# Gold, surely!
# But, wonderful to relate, not an irregular, shapeless fragment of crude ore, fresh from Nature's crucible, but a bit of jeweler's handicraft in the form of a plain gold ring.
# Looking at it more attentively, he saw that it bore the inscription, "May to Cass."

*** Reading from input files ***
  %s sample_text.txt
[CLS] wood pile refused to kind ##le a fire to dry his bed [MASK] clothes , and he had [MASK] ##ours ##e [MASK] a more provide ##nt [MASK] ' s to supply the deficiency . 
this was nearly opposite . 
mr . 
cass ##ius crossed the highway , and stopped [MASK] . 
something [MASK] ##ed in the nearest [MASK] pool before him . 
gold [SEP] wonderful to relate , not [MASK] irregular , shape ##less fragment of crude ore , fresh [MASK] nature ' [MASK] cr ##ucible , but a bit of [MASK] ##er ' s hand ##ic ##raf ##t in [MASK] form of a [MASK] gold ring . 
[MASK] at it more at ##ten ##tively [MASK] [MASK] saw that it bore the inscription , " may [SEP] 